In [2]:
import os.path, csv, glob
import import_ipynb
from keras.models import Input
from keras.preprocessing import image
from Extractor import Extractor
import tensorflow as tf
import numpy as np
import loupe_keras as lpk
from Model import MyNet
from keras.applications.inception_v3 import InceptionV3, preprocess_input
def getFrames(sample):
        root_path = 'SoccerNet/SoccerNet-code/data/data'  
        code_action = sample[0][:3]
        game_time = sample[0][3]+'_'+sample[0][6:11]   
        game_code = sample[2]        
        name_file = code_action+game_time
        specific_path =os.path.join(code_action,game_code,game_time)
        path = os.path.join(root_path, specific_path)         
        images = sorted(glob.glob(os.path.join(path, name_file+'*jpg')))         
        return images
    

    
def rescaleList(input_list, size):        
        assert len(input_list) >= size
        skip = len(input_list) // size
        output = [input_list[i] for i in range(0, len(input_list), skip)]
        return output[:size]
    
sequence_length = 100 
exmodel= Extractor()
trainer = MyNet()
with open('data_file.csv','r') as datafile:
    reader = csv.reader(datafile, delimiter=',')
    globalList=[]
    for row in reader:
        frames = getFrames(row)        
        frames = rescaleList(frames, sequence_length)
        sequence =np.empty((0,2048), dtype=object)
        for image in frames:            
            features= exmodel.extract(image) #restituisce un [(2048)]
            features = tf.expand_dims(features, axis=0) 
            sequence = tf.concat( (sequence, features), axis=0)
        trainer.modello.predict(sequence)
        
       

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100, 2048)         0         
_________________________________________________________________
net_vlad_3 (NetVLAD)         (None, 72)                9699392   
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 219       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 4         
Total params: 9,699,615
Trainable params: 9,699,615
Non-trainable params: 0
_________________________________________________________________
None


ValueError: If your data is in the form of symbolic tensors, you should specify the `steps` argument (instead of the `batch_size` argument, because symbolic tensors are expected to produce batches of input data).